In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2

In [2]:

path = r'C:\Users\fajar.fatoni\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK067656,Karma Resorts,Other,Past Guests,Karma Kandara,PF299367,Mr,Robyn,Whalan,Male,...,2025-09-15 00:00:00,2,OTA,OTA,RETAIL,1,0,0,Phone Only,NO
1,KK067377,Karma Resorts,Other,Past Guests,Karma Kandara,PF298764,Mr,Michael,Ference,Male,...,2025-09-12 00:00:00,7,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
2,KK065630,Karma Resorts,Other,Past Guests,Karma Kandara,PF295943,Mr,Ashley,James,Male,...,2025-09-15 00:00:00,2,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
3,KK074130,Karma Resorts,Other,Past Guests,Karma Kandara,PF125748,Mr,Kevin,Matthews,Male,...,2025-09-14 00:00:00,13,HOUSE_USE,V_OWNER,COMP,6,0,0,Email and Phone,NO
4,KK078688,Karma Resorts,Other,Past Guests,Karma Kandara,PF326009,Miss,Stephanie,Fu,Female,...,2025-09-09 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,5,0,1,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,KSM016767,Karma Resorts,Other,Past Guests,Karma St Martins,PF064162,Miss,Jane,Cooper,Unknown,...,2025-09-12 00:00:00,2,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
95,KLM008743,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF307670,Ms,Lisa,Harland (Bride),Female,...,2025-09-11 00:00:00,0,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
96,KB052573,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325196,Mrs,Sabrina,Mccracken,Female,...,2025-09-07 00:00:00,6,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
97,KLM005986,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF279614,Ms,Lois,Cole,Female,...,2025-09-11 00:00:00,2,OTA,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar.fatoni\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:127: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [7]:
df_analytic.loc[
    (df_analytic['TelNo'] == 'empty') & (df_analytic['MobileNo'] != 'empty'), 
    'TelNo'] = df_analytic['MobileNo']
df_analytic.loc[df_analytic['TelNo'] == df_analytic['MobileNo'],'MobileNo'] = 'empty'

In [8]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
8,KK069664,Karma Resorts,Other,Past Guests,Karma Kandara,PF299367,Mr,Robyn,Whalan,Male,...,2025-09-15 00:00:00,2,OTA,OTA,RETAIL,1,0,0,Phone Only,NO
11,KK069249,Karma Resorts,Other,Past Guests,Karma Kandara,PF302743,Miss,Ying,Luo,Female,...,2025-09-08 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,5,0,0,Email and Phone,NO
19,KB051787,Karma Resorts,Other,Past Guests,Karma Bavaria,PF320898,empty,Hendrik,Urban,Unknown,...,2025-09-13 00:00:00,1,OTA,OTA,RETAIL,2,2,0,Phone Only,NO
58,KSM012546,Karma Resorts,Other,Past Guests,Karma St Martins,PF270569,Dr,Arlene,Vetere,Unknown,...,2025-09-11 00:00:00,6,DIR,TEL,RETAIL,1,0,0,Email and Phone,NO
78,KLM010982,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF318378,Mr,Agency,Chef,Male,...,2025-09-08 00:00:00,3,COMP,STAFF,COMP,2,0,0,Email and Phone,NO
82,KSM016868,Karma Resorts,Other,Past Guests,Karma St Martins,PF071366,Mr,Richard,Larn,Unknown,...,2025-09-11 00:00:00,10,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
88,KSM014817,Karma Resorts,Other,Past Guests,Karma St Martins,PF293853,Mr,Michael,Roberts,Unknown,...,2025-09-08 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
94,KSM016767,Karma Resorts,Other,Past Guests,Karma St Martins,PF064162,Miss,Jane,Cooper,Unknown,...,2025-09-12 00:00:00,2,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO


In [9]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK067656,Karma Resorts,Other,Past Guests,Karma Kandara,PF299367,Mr,Robyn,Whalan,Male,...,2025-09-15 00:00:00,2,OTA,OTA,RETAIL,1,0,0,Phone Only,NO
1,KK067377,Karma Resorts,Other,Past Guests,Karma Kandara,PF298764,Mr,Michael,Ference,Male,...,2025-09-12 00:00:00,7,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
2,KK065630,Karma Resorts,Other,Past Guests,Karma Kandara,PF295943,Mr,Ashley,James,Male,...,2025-09-15 00:00:00,2,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
3,KK074130,Karma Resorts,Other,Past Guests,Karma Kandara,PF125748,Mr,Kevin,Matthews,Male,...,2025-09-14 00:00:00,13,HOUSE_USE,V_OWNER,COMP,6,0,0,Email and Phone,NO
4,KK078688,Karma Resorts,Other,Past Guests,Karma Kandara,PF326009,Miss,Stephanie,Fu,Female,...,2025-09-09 00:00:00,5,HOUSE_USE,V_OWNER_NRB,COMP,5,0,1,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,KSM016814,Karma Resorts,Other,Past Guests,Karma St Martins,PF326770,Mrs,Julia,Walder,Unknown,...,2025-09-08 00:00:00,2,COMP,TEL,RETAIL,2,0,0,Email and Phone,NO
87,KLM008743,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF307670,Ms,Lisa,Harland (Bride),Female,...,2025-09-11 00:00:00,0,DIR,WEDDING,GROUP,2,0,0,Email and Phone,NO
88,KB052573,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325196,Mrs,Sabrina,Mccracken,Female,...,2025-09-07 00:00:00,6,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
89,KLM005986,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF279614,Ms,Lois,Cole,Female,...,2025-09-11 00:00:00,2,OTA,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO


In [10]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [11]:
find_member_booking.to_csv(r'C:\Users\fajar.fatoni\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [12]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
3,KK074130,Karma Resorts,Other,Past Guests,Karma Kandara,PF125748,Mr,Kevin,Matthews,Male,...,2025-09-14 00:00:00,13,HOUSE_USE,V_OWNER,COMP,6,0,0,Email and Phone,NO
22,KB045760,Karma Resorts,Other,Past Guests,Karma Bavaria,PF147772,Frau,Claudia,Goebl,Female,...,2025-09-13 00:00:00,4,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
24,KB052437,Karma Resorts,Other,Past Guests,Karma Bavaria,PF264770,Mr,Timotheus Matthaus,Stroebel,Male,...,2025-09-07 00:00:00,1,COMP,GER_OWE,COMP,1,0,0,Email and Phone,NO
27,KK068861,Karma Resorts,Other,Past Guests,Karma Kandara,PF299103,Miss,Caroline,Impler,Female,...,2025-09-12 00:00:00,1,HOUSE_USE,GER_OWE,COMP,3,0,0,Email and Phone,NO
28,KB052801,Karma Resorts,Other,Past Guests,Karma Bavaria,PF329097,Herr,Matthias,Nees,Male,...,2025-09-11 00:00:00,4,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
34,KB052800,Karma Resorts,Other,Past Guests,Karma Bavaria,PF072877,Herr,Dieter,Nees,Male,...,2025-09-11 00:00:00,4,COMP,GER_OWE,COMP,2,0,0,Email and Phone,NO
43,KB050114,Karma Resorts,Other,Past Guests,Karma Bavaria,PF312197,Ms,Tatiana,Klatis,Female,...,2025-09-08 00:00:00,4,COMP,GER_OWE,COMP,1,0,0,Email Only,NO
53,KSM015784,Karma Resorts,Other,Past Guests,Karma St Martins,PF067424,Mr,Karma St. Martin'S,Management,Male,...,2025-09-13 00:00:00,24,COMP,STAFF,COMP,2,0,0,Email and Phone,NO
86,KSM016814,Karma Resorts,Other,Past Guests,Karma St Martins,PF326770,Mrs,Julia,Walder,Unknown,...,2025-09-08 00:00:00,2,COMP,TEL,RETAIL,2,0,0,Email and Phone,NO


In [13]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [14]:
count_data_akhir = df_analytic.shape[0]

In [15]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB053296,Karma Resorts,Other,Past Guests,Karma Bavaria,PF329026,Herr,Haarer,Klaus,Male,...,2025-09-07 00:00:00,1,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
1,KB053284,Karma Resorts,Other,Past Guests,Karma Bavaria,PF328977,Herr,Waldemar,Fiedler,Unknown,...,2025-09-07 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
2,KB052589,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325246,Frau,Charlotte,Merkel,Female,...,2025-09-07 00:00:00,2,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
3,KB052704,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325826,Herr,Juergen,Za,Unknown,...,2025-09-07 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
4,KSM016665,Karma Resorts,Other,Past Guests,Karma St Martins,PF245884,empty,Aurelien,Prunier,Male,...,2025-09-07 00:00:00,3,OTA,OTA,RETAIL,3,0,0,Phone Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,KLM011039,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF318378,Mr,Agency,Chef,Male,...,2025-09-14 00:00:00,2,COMP,STAFF,COMP,2,0,0,Email and Phone,NO
78,KSM014740,Karma Resorts,Other,Past Guests,Karma St Martins,PF303281,Mr,& Mrs,Phillipson,Unknown,...,2025-09-15 00:00:00,2,DIR,TEL,RETAIL,2,0,0,Phone Only,NO
79,KK065472,Karma Resorts,Other,Past Guests,Karma Kandara,PF295555,Mr.,Gordon Douglas,Mckendry,Male,...,2025-09-15 00:00:00,7,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
80,KK065630,Karma Resorts,Other,Past Guests,Karma Kandara,PF295943,Mr,Ashley,James,Male,...,2025-09-15 00:00:00,2,OTA,OTA,RETAIL,4,0,0,Phone Only,NO


In [16]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [17]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar.fatoni\AppData\Local\Temp\ipykernel_16440\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [18]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB053296,Karma Resorts,Other,Past Guests,Karma Bavaria,PF329026,Herr,Haarer,Klaus,Male,...,2025-09-07,1,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
1,KLM007759,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF297805,Dhr.,Danny,Meynen,Unknown,...,2025-09-07,2,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
2,KSM016532,Karma Resorts,Other,Past Guests,Karma St Martins,PF324208,Ms,Katharine,Rowland,Unknown,...,2025-09-07,2,DIR,GHPA,DIGITAL,2,0,0,Email and Phone,NO
3,KB052573,Karma Resorts,Other,Past Guests,Karma Bavaria,PF325196,Mrs,Sabrina,Mccracken,Female,...,2025-09-07,6,WYNDHAM,WYNDHAM,WYNDHAM,2,0,0,Email and Phone,NO
4,KB047576,Karma Resorts,Other,Past Guests,Karma Bavaria,PF301742,,Astrid,Schuderer,Unknown,...,2025-09-07,2,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,KLM011039,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF318378,Mr,Agency,Chef,Male,...,2025-09-14,2,COMP,STAFF,COMP,2,0,0,Email and Phone,NO
78,KK065630,Karma Resorts,Other,Past Guests,Karma Kandara,PF295943,Mr,Ashley,James,Male,...,2025-09-15,2,OTA,OTA,RETAIL,4,0,0,Phone Only,NO
79,KSM014740,Karma Resorts,Other,Past Guests,Karma St Martins,PF303281,Mr,& Mrs,Phillipson,Unknown,...,2025-09-15,2,DIR,TEL,RETAIL,2,0,0,Phone Only,NO
80,KK065472,Karma Resorts,Other,Past Guests,Karma Kandara,PF295555,Mr.,Gordon Douglas,Mckendry,Male,...,2025-09-15,7,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO


In [19]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 99
Jumlah Data Yang Duplicate : 8
Jumlah Data Setelah Hapus Duplicate : 91
Jumlah Data Yang Termasuk Member: 9
Jumlah Data Setelah Hapus Member : 82
Jumlah Data Akhir : 82


In [20]:
df_analytic.to_excel(r'C:\Users\fajar.fatoni\Documents\Python\Data\final_analytics.xlsx', index=False)